In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
# import transfer_func_util as tf
# import configuration as config

### Discharge data

In [21]:
i = 0
fname = r'/data/old_trap/20210603/bead1/test/turbombar_powfb_xyzcool_elec5_200mV41Hz.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
_,ax = plt.subplots()
ax.scatter(range(5000), bb.other_data[7,:5000])
len(bb.other_data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

8

In [23]:
get_amp(0)

6959.377652545751

In [22]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
fsamp = 5000
def get_amp(bb):
    
#     fname = r'/data/old_trap/20210530/bead1/neutrality/w_height_fb_100V_y_sweep_spin_xz_100kHz/turbombar_powfb_xyzcool_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 'limit_f': [0, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[7])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
#     freq2 = np.fft.rfftfreq(len(bb.other_data[6]), d=1./5000)
#     fft_angles = np.angle(np.fft.rfft(bb.other_data[6]))

    phi_tmp = m1_tmp.values[2]
#     phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*50, np.std(xx2)*np.sqrt(2)*50)
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-100000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]